In [0]:
# Re-executar a criação do DataFrame (se você estiver em uma nova célula ou sessão)
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

data = [
  ("Alice", 1, 1.70, "Engenharia"),
  ("Bob", 2, 1.85, "Ciência da Computação"),
  ("Charlie", 3, 1.75, "Engenharia"),
  ("David", 4, 1.90, "Matemática"),
  ("Eve", 5, 1.60, "Ciência da Computação"),
  ("Frank", 6, None, "Engenharia")
]

schema = StructType([
  StructField("Nome", StringType(), True),
  StructField("ID", IntegerType(), True),
  StructField("Altura", DoubleType(), True),
  StructField("Departamento", StringType(), True)
])

df = spark.createDataFrame(data, schema)

# Criar uma view temporária para usar com Spark SQL
df.createOrReplaceTempView("pessoas")

print("DataFrame original:")
df.show()
print("\nView temporária 'pessoas' criada.")

### a) Seleção e Renomeação de Colunas (Spark SQL):

In [0]:
%sql
-- Selecionar apenas algumas colunas
SELECT Nome, Departamento FROM pessoas;

-- Renomear uma coluna
SELECT Nome, ID, Altura, Departamento AS Area FROM pessoas;

-- Selecionar e renomear ao mesmo tempo
SELECT Nome AS Nome_Completo, ID, Altura AS Altura_Metros FROM pessoas;

### b) Filtragem de Dados (Spark SQL):

In [0]:
%sql
-- Filtrar por uma condição (pessoas da Engenharia)
SELECT * FROM pessoas WHERE Departamento = 'Engenharia';

-- Filtrar por múltiplas condições (ID > 3 E Altura > 1.70)
SELECT * FROM pessoas WHERE ID > 3 AND Altura > 1.70;

-- Filtrar por Altura > 1.80
SELECT * FROM pessoas WHERE Altura > 1.80;

### c) Criação e Modificação de Colunas (Spark SQL):

In [0]:
%sql
-- Adicionar uma nova coluna com valor constante
--SELECT *, 'Brasil' AS Pais FROM pessoas;

-- Criar uma nova coluna baseada em uma condição (ex: 'Status_Altura')
/*SELECT
*,
CASE
  WHEN Altura >= 1.80 THEN 'Alto'
  WHEN Altura < 1.70 THEN 'Baixo'
  ELSE 'Medio'
END AS Status_Altura
FROM pessoas;*/

-- Modificar uma coluna existente (ex: Altura em cm)
SELECT Nome, ID, Altura * 100 AS Altura_cm, Departamento FROM pessoas;

### d) Agregações (Spark SQL):

In [0]:
%sql
-- Contar o número de pessoas por departamento
--SELECT Departamento, COUNT(*) AS Contagem FROM pessoas GROUP BY Departamento;

-- Calcular a média da altura por departamento
--SELECT Departamento, AVG(Altura) AS Media_Altura FROM pessoas GROUP BY Departamento;

-- Múltiplas agregações
SELECT
Departamento,
AVG(Altura) AS Media_Altura,
MIN(Altura) AS Min_Altura,
MAX(Altura) AS Max_Altura
FROM pessoas
GROUP BY Departamento;

### e) Ordenação (Spark SQL):

In [0]:
%sql
-- Ordenar por Altura (crescente)
SELECT * FROM pessoas ORDER BY Altura ASC;

-- Ordenar por Departamento (crescente) e depois por Altura (decrescente)
SELECT * FROM pessoas ORDER BY Departamento ASC, Altura DESC;

### f) Remoção de Duplicatas (Spark SQL):

In [0]:
# Adicionar um dado duplicado para demonstração
data_duplicado = data + [("Alice", 1, 1.70, "Engenharia")]
df_com_duplicata = spark.createDataFrame(data_duplicado, schema)
df_com_duplicata.createOrReplaceTempView("pessoas_com_duplicata")

print("\nDataFrame com duplicata (para SQL):")
df_com_duplicata.show()

In [0]:
%sql
-- Remover linhas duplicadas (considera todas as colunas)
--SELECT DISTINCT * FROM pessoas_com_duplicata;

-- Remover duplicatas com base em um subconjunto de colunas (ex: Nome e Departamento)
-- Isso é um pouco mais complexo em SQL puro sem funções de janela, mas pode ser feito com GROUP BY
-- ou com funções de janela (ROW_NUMBER) para selecionar a primeira ocorrência.
-- Usando GROUP BY para simular a remoção de duplicatas em um subconjunto:
/*SELECT Nome, ID, Altura, Departamento
FROM (
SELECT *,
       ROW_NUMBER() OVER (PARTITION BY Nome, Departamento ORDER BY ID) as rn
FROM pessoas_com_duplicata
)
WHERE rn = 1;*/

-- Ou, se você quer apenas as colunas Nome e Departamento sem duplicatas:
SELECT DISTINCT Nome, Departamento FROM pessoas_com_duplicata;

### g) Tratamento de Valores Nulos (Spark SQL):

In [0]:
%sql
-- DataFrame com valor nulo (Frank tem Altura nula)
-- A view 'pessoas' já tem o valor nulo para Frank.

-- Remover linhas com qualquer valor nulo
--SELECT * FROM pessoas WHERE Altura IS NOT NULL AND Nome IS NOT NULL AND ID IS NOT NULL AND Departamento IS NOT NULL;
-- Ou, mais conciso se você sabe quais colunas podem ter nulos:
--SELECT * FROM pessoas WHERE Altura IS NOT NULL;

-- Preencher valores nulos em uma coluna específica
/*SELECT
Nome,
ID,
COALESCE(Altura, 0.0) AS Altura, -- Substitui NULL por 0.0 na coluna Altura
Departamento
FROM pessoas;*/

-- Preencher valores nulos em todas as colunas do mesmo tipo (ex: Departamento)
SELECT
Nome,
ID,
Altura,
COALESCE(Departamento, 'Desconhecido') AS Departamento
FROM pessoas;


In [0]:
# Exemplo de como executar SQL e obter um DataFrame PySpark
df_engenharia_sql = spark.sql("SELECT * FROM pessoas WHERE Departamento = 'Engenharia'")
print("\nResultado da query SQL em um DataFrame PySpark:")
df_engenharia_sql.show()

In [0]:
# Importar tipos de dados do PySpark
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

# DataFrame de Pessoas (usando os dados anteriores)
data_pessoas = [
("Alice", 1, 1.70, "Engenharia"),
("Bob", 2, 1.85, "Ciência da Computação"),
("Charlie", 3, 1.75, "Engenharia"),
("David", 4, 1.90, "Matemática"),
("Eve", 5, 1.60, "Ciência da Computação"),
("Frank", 6, None, "Engenharia"),
("Grace", 7, 1.68, "Marketing") # Adicionando um novo para demonstração de joins
]

schema_pessoas = StructType([
StructField("Nome", StringType(), True),
StructField("ID", IntegerType(), True),
StructField("Altura", DoubleType(), True),
StructField("Departamento", StringType(), True)
])

df_pessoas = spark.createDataFrame(data_pessoas, schema_pessoas)
df_pessoas.createOrReplaceTempView("pessoas") # Criar view temporária para SQL

print("DataFrame de Pessoas:")
df_pessoas.show()

# DataFrame de Departamentos
data_departamentos = [
("Engenharia", "ENG", "São Paulo"),
("Ciência da Computação", "COMP", "Rio de Janeiro"),
("Matemática", "MAT", "Belo Horizonte"),
("Recursos Humanos", "RH", "São Paulo") # Departamento que não tem pessoas ainda
]

schema_departamentos = StructType([
StructField("Nome_Departamento", StringType(), True),
StructField("Sigla_Departamento", StringType(), True),
StructField("Localizacao", StringType(), True)
])

df_departamentos = spark.createDataFrame(data_departamentos, schema_departamentos)
df_departamentos.createOrReplaceTempView("departamentos") # Criar view temporária para SQL

print("\nDataFrame de Departamentos:")
df_departamentos.show()

### a) INNER JOIN (Junção Interna)

O que faz: Retorna apenas as linhas onde há correspondência em ambos os DataFrames/tabelas.
Quando usar: Quando você só quer os registros que têm informações em todas as fontes.

In [0]:
# INNER JOIN
df_inner_join = df_pessoas.join(df_departamentos, df_pessoas.Departamento == df_departamentos.Nome_Departamento, "inner")
print("\nINNER JOIN:")
df_inner_join.show()

In [0]:
%sql
-- INNER JOIN
SELECT p.Nome, p.Departamento, d.Sigla_Departamento, d.Localizacao
FROM pessoas p
INNER JOIN departamentos d ON p.Departamento = d.Nome_Departamento;

###b) LEFT (OUTER) JOIN (Junção Externa Esquerda)

O que faz: Retorna todas as linhas do DataFrame/tabela da esquerda e as linhas correspondentes do DataFrame/tabela da direita. Se não houver correspondência na direita, os valores serão null.
Quando usar: Quando você quer manter todos os registros da tabela principal (esquerda) e adicionar informações da tabela secundária (direita, se houver).

In [0]:
# LEFT JOIN
df_left_join = df_pessoas.join(df_departamentos, df_pessoas.Departamento == df_departamentos.Nome_Departamento, "left_outer")
print("\nLEFT JOIN:")
df_left_join.show()

In [0]:
%sql
-- LEFT JOIN
SELECT p.Nome, p.Departamento, d.Sigla_Departamento, d.Localizacao
FROM pessoas p
LEFT JOIN departamentos d ON p.Departamento = d.Nome_Departamento;

### c) RIGHT (OUTER) JOIN (Junção Externa Direita)

O que faz: Retorna todas as linhas do DataFrame/tabela da direita e as linhas correspondentes do DataFrame/tabela da esquerda. Se não houver correspondência na esquerda, os valores serão null.
Quando usar: Quando você quer manter todos os registros da tabela secundária (direita) e adicionar informações da tabela principal (esquerda, se houver).

In [0]:
# RIGHT JOIN
df_right_join = df_pessoas.join(df_departamentos, df_pessoas.Departamento == df_departamentos.Nome_Departamento, "right_outer")
print("\nRIGHT JOIN:")
df_right_join.show()

In [0]:
%sql
-- RIGHT JOIN
SELECT p.Nome, p.Departamento, d.Sigla_Departamento, d.Localizacao
FROM pessoas p
RIGHT JOIN departamentos d ON p.Departamento = d.Nome_Departamento;

### d) FULL (OUTER) JOIN (Junção Externa Completa)

O que faz: Retorna todas as linhas de ambos os DataFrames/tabelas. Se não houver correspondência em um dos lados, os valores serão null para as colunas desse lado.
Quando usar: Quando você quer ver todos os registros de ambas as tabelas, independentemente de haver correspondência.

In [0]:
# FULL JOIN
df_full_join = df_pessoas.join(df_departamentos, df_pessoas.Departamento == df_departamentos.Nome_Departamento, "full_outer")
print("\nFULL JOIN:")
df_full_join.show()

In [0]:
%sql
-- FULL JOIN
SELECT p.Nome, p.Departamento, d.Sigla_Departamento, d.Localizacao
FROM pessoas p
FULL JOIN departamentos d ON p.Departamento = d.Nome_Departamento;

### e) ANTI JOIN (Junção Anti)

O que faz: Retorna as linhas do DataFrame/tabela da esquerda que não têm correspondência no DataFrame/tabela da direita. É o oposto de um INNER JOIN para o lado esquerdo.
Quando usar: Para encontrar registros "órfãos" ou identificar dados que não existem em outra tabela.

In [0]:
# ANTI JOIN
df_anti_join = df_pessoas.join(df_departamentos, df_pessoas.Departamento == df_departamentos.Nome_Departamento, "left_anti")
print("\nANTI JOIN (Pessoas sem Departamento correspondente na tabela de Departamentos):")
df_anti_join.show()

In [0]:
%sql
-- ANTI JOIN (usando NOT EXISTS ou LEFT JOIN com WHERE IS NULL)
-- Opção 1: Usando NOT EXISTS
SELECT p.Nome, p.Departamento
FROM pessoas p
WHERE NOT EXISTS (
SELECT 1 FROM departamentos d WHERE p.Departamento = d.Nome_Departamento
);

-- Opção 2: Usando LEFT JOIN com WHERE IS NULL (comum para simular ANTI JOIN)
SELECT p.Nome, p.Departamento
FROM pessoas p
LEFT JOIN departamentos d ON p.Departamento = d.Nome_Departamento
WHERE d.Nome_Departamento IS NULL;